## Importing libraries

In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

## Functions

In [2]:
# Input unction
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

# Function to split data into batches of a given size
def split_into_batches(df, batch_size=100000):
    nb_rows = len(df.index)
    intervals = []
  
    for i in range(0, nb_rows + 1, batch_size):
        intervals.append(i)
  
    if(intervals[-1] != nb_rows):
        intervals.append(nb_rows)
  
    batches_X = []
    batches_y = []
  
    for i in range(0, len(intervals) - 1):
        batches_X.append(train.iloc[intervals[i]:intervals[i + 1], :][features])
        batches_y.append(train.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

    return batches_X, batches_y

## Data

In [3]:
path = "Data/CSV_FISCHER"

files = glob.glob(path + "/*.csv")
         
array = []

# Create one single file and preparing the dataset for the training
for i in files:
    df = pd.read_csv(i, index_col=None, header=0)
    array.append(df)

train = pd.concat(array, axis=0, ignore_index=True)
train = shuffle(train)
features = list(train.iloc[:, 0:192].columns)
x = train[features]
y = train["good_move"]

# Specifying the feature columns
categoricalColumns = list(x.iloc[:, 0:63].columns)
numericalColumns = list(x.iloc[:, 64:192].columns)
featureColumns = []

for feature_name in categoricalColumns:
    vocabulary = x[feature_name].unique()
    featureColumns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in numericalColumns:
    featureColumns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

# Splitting the dataset into two batches
batches_x, batches_y = split_into_batches(train)

## Model

In [4]:
estimator = tf.estimator.LinearClassifier(feature_columns = featureColumns)
train_input_fns = []

for df_x, df_y in zip(batches_x, batches_y):
    train_input_fns.append(make_input_fn(df_x, df_y))

# Training the model on all the input functions
i = 1
for train_input_fn in train_input_fns:
    i = i + 1
    estimator.train(train_input_fn)

# Saving the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(tf.feature_column.make_parse_example_spec(featureColumns))
pathModel = estimator.export_saved_model(r"C:\Users\giaco\OneDrive\Desktop\Informatica\Master\Codes, Projects and Researches\AI Chess\Model", serving_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\giaco\\AppData\\Local\\Temp\\tmp99f405ys', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_v

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 30.6981
INFO:tensorflow:loss = 0.025088945, step = 100 (3.258 sec)
INFO:tensorflow:global_step/sec: 155.551
INFO:tensorflow:loss = 0.14220726, step = 200 (0.645 sec)
INFO:tensorflow:global_step/sec: 161.419
INFO:tensorflow:loss = 0.24283797, step = 300 (0.618 sec)
INFO:tensorflow:global_step/sec: 161.25
INFO:tensorflow:loss = 0.15411615, step = 400

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt-31250
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 31250...
INFO:tensorflow:Saving checkpoints for 31250 into C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 31250...
INFO:tensorflow:loss = 0.03374291, step = 31250
INFO:tensorflow:global_step/sec: 33.8999
INFO:tensorflow:loss = 0.029682804, step = 31350 (2.950 sec)
INFO:tensorflow:global_step/sec: 158.795
INFO:tensorflow:loss = 0.027531033, step = 31450 (0.630 sec)
INFO:tensorflow:global_step/sec: 153.715
INFO:tensorflow:loss = 0.17710975, step = 31550 (0.651 sec)
INFO

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt-187500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 187500...
INFO:tensorflow:Saving checkpoints for 187500 into C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 187500...
INFO:tensorflow:loss = 0.13957456, step = 187500
INFO:tensorflow:global_step/sec: 25.9775
INFO:tensorflow:loss = 0.31073344, step = 187600 (3.839 sec)
INFO:tensorflow:global_step/sec: 128.051
INFO:tensorflow:loss = 0.14520101, step = 187700 (0.790 sec)
INFO:tensorflow:global_step/sec: 123.797
INFO:tensorflow:loss = 0.12292333, step = 187800 (0.801 sec)
INFO:tensorflow:global_step/sec: 124.733
INFO:tensorflow:loss = 0.35

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt-218750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 218750...
INFO:tensorflow:Saving checkpoints for 218750 into C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 218750...
INFO:tensorflow:loss = 0.23089138, step = 218750
INFO:tensorflow:global_step/sec: 24.5958
INFO:tensorflow:loss = 0.3282953, step = 218850 (4.066 sec)
INFO:tensorflow:global_step/sec: 107.742
INFO:tensorflow:loss = 0.22142348, step = 218950 (0.929 sec)
INFO:tensorflow:global_step/sec: 107.432
INFO:tensorflow:loss = 0.124892294, step = 219050 (0.929 sec)
INFO:tensorflow:global_step/sec: 110.971
INFO:tensorflow:loss = 0.12

C:\Users\giaco\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt-250000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 250000...
INFO:tensorflow:Saving checkpoints for 250000 into C:\Users\giaco\AppData\Local\Temp\tmp99f405ys\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 250000...
INFO:tensorflow:loss = 0.03143397, step = 250000
INFO:tensorflow:global_step/sec: 29.268
INFO:tensorflow:loss = 0.033810478, step = 250100 (3.417 sec)
INFO:tensorflow:global_step/sec: 125.915
INFO:tensorflow:loss = 0.033483453, step = 250200 (0.795 sec)
INFO:tensorflow:global_step/sec: 117.796
INFO:tensorflow:loss = 0.24618682, step = 250300 (0.849 sec)
INFO:tensorflow:global_step/sec: 120.18
INFO:tensorflow:loss = 0.03